# Ejemplo Profesional de GridSearchCV

Este notebook demuestra cómo utilizar `GridSearchCV` de una manera robusta y profesional, siguiendo las mejores prácticas:

1.  **Uso de Pipelines**: Para evitar el *data leakage* (fuga de datos) durante el preprocesamiento (ej. escalado).
2.  **Validación Cruzada Estratificada**: Para mantener la proporción de clases en cada fold.
3.  **Análisis de Resultados**: Visualización de los puntajes y tiempos de entrenamiento.
4.  **Evaluación Final**: Uso de un conjunto de prueba separado (hold-out set).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Configuración de estilo para gráficos
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# Cargar datos
data = load_breast_cancer()
X = data.data
y = data.target

# Crear un DataFrame para visualizar mejor (opcional)
df = pd.DataFrame(X, columns=data.feature_names)
df['target'] = y
print(f"Dimensiones del dataset: {df.shape}")
df.head()



In [ ]:
# División Train/Test
# Es CRÍTICO separar un conjunto de test que NUNCA vea el GridSearch
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Entrenamiento: {X_train.shape}, Test: {X_test.shape}")

## 2. Construcción del Pipeline
El uso de `Pipeline` es esencial. Si escalamos los datos ANTES de dividir en CV, introducimos *data leakage* (el modelo "ve" estadísticas del conjunto de validación). El pipeline asegura que el escalado se ajuste solo en los datos de entrenamiento de cada fold.

In [ ]:
# Definimos los pasos: Escalado -> Modelo (SVM)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(probability=True)) # probability=True para poder usar curvas ROC si fuera necesario
])

pipeline

## 3. Definición de la Grilla de Hiperparámetros
Especificamos los parámetros a probar. Note la sintaxis `nombre_paso__parametro`.

In [ ]:
param_grid = [
    # Opción 1: Kernel RBF
    {
        'svm__C': [0.1, 1, 10, 100],
        'svm__gamma': [0.001, 0.01, 0.1, 1],
        'svm__kernel': ['rbf']
    },
    # Opción 2: Kernel Lineal (gamma no aplica)
    {
        'svm__C': [0.1, 1, 10, 100],
        'svm__kernel': ['linear']
    }
]

param_grid

## 4. Ejecución de GridSearchCV
Configuramos la búsqueda con validación cruzada estratificada y métrica de scoring.

In [ ]:
# StratifiedKFold asegura que cada fold tenga la misma proporción de clases
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=cv,
    scoring='accuracy', # O 'f1', 'roc_auc', etc.
    n_jobs=-1,          # Usar todos los núcleos disponibles
    verbose=1,
    return_train_score=True # Útil para diagnosticar overfitting
)

# Entrenar
print("Iniciando búsqueda de hiperparámetros...")
grid_search.fit(X_train, y_train)

print(f"\nMejores parámetros encontrados: {grid_search.best_params_}")
print(f"Mejor score de validación cruzada: {grid_search.best_score_:.4f}")

## 5. Análisis de Resultados
Convertimos los resultados a un DataFrame para inspeccionarlos visualmente.

In [ ]:
results_df = pd.DataFrame(grid_search.cv_results_)

# Filtramos columnas interesantes
cols_to_show = ['param_svm__kernel', 'param_svm__C', 'param_svm__gamma', 'mean_test_score', 'std_test_score', 'rank_test_score']
results_df[cols_to_show].sort_values(by='rank_test_score').head(10)


In [ ]:
# Visualización: Heatmap para el kernel RBF
# Filtramos solo los resultados de RBF para poder hacer una matriz 2D (C vs Gamma)
rbf_results = results_df[results_df['param_svm__kernel'] == 'rbf']

pivot_table = rbf_results.pivot(index='param_svm__C', columns='param_svm__gamma', values='mean_test_score')

plt.figure(figsize=(8, 6))
sns.heatmap(pivot_table, annot=True, cmap='viridis', fmt='.3f')
plt.title('Accuracy en Validación Cruzada (Kernel RBF)')
plt.xlabel('Gamma')
plt.ylabel('C')
plt.show()

## 6. Evaluación Final en Test Set
Finalmente, evaluamos el mejor modelo (que `GridSearchCV` re-entrenó automáticamente en todo `X_train` gracias a `refit=True`) en el conjunto de prueba reservado.

In [ ]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

# Matriz de Confusión
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=data.target_names)
disp.plot(cmap='Blues')
plt.title('Matriz de Confusión en Test Set')
plt.grid(False)
plt.show()